In [1]:
import gensim
import os
import collections
import random
import csv
import sys

In [2]:
csv.field_size_limit(sys.maxsize)

131072

In [3]:
corpusfile = 'tokenized.txt'
modelfile = 'aozora2vec.model'

In [4]:
def read_corpus():
    with open(corpusfile, newline='', encoding='utf-8') as csvfile:
        for index, tokens_text in enumerate(csvfile):
            tokens = gensim.utils.to_unicode(tokens_text).split()
            yield gensim.models.doc2vec.TaggedDocument(tokens, [index])

In [5]:
corpus = list(read_corpus())

In [6]:
model = gensim.models.doc2vec.Doc2Vec(size=100, min_count=5, iter=50, workers=36)

In [7]:
model.build_vocab(corpus)

In [8]:
import resource
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

10926840

In [9]:
%time model.train(corpus)

CPU times: user 9h 4min 33s, sys: 6.62 s, total: 9h 4min 39s
Wall time: 31min 22s


2891128200

In [10]:
model.save('aozora2vec-iter50.model')

In [11]:
ranks = []
second_ranks = []
for doc_id in range(1000):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(corpus[doc_id].tags[0])
    ranks.append(rank)
    second_ranks.append(sims[1])

In [12]:
collections.Counter(ranks)

Counter({0: 935,
         1: 41,
         2: 3,
         3: 3,
         4: 3,
         6: 1,
         8: 1,
         9: 3,
         10: 1,
         13: 1,
         14: 1,
         19: 1,
         23: 1,
         24: 1,
         37: 1,
         79: 1,
         86: 1,
         5188: 1})